In [144]:
import numpy as np
import pandas as pd
from astropy.table import Table
from astropy.io.fits import getdata
from astropy.time import Time
from astropy.io import fits
import sys
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
import astropy.units as u



sc2 = Table.read('fits/straycats_v2_incr2.fits', format = 'fits')

for key in sc2.columns:
    if sc2[key].dtype.str.startswith('|S'):
        sc2[key] = sc2[key].astype('str') 
df = sc2.to_pandas()
df.columns
meta2 = Table.read('fits/StrayCats_SLsources_3arcminbkg.fits')

for key in meta2.columns:
    if meta2[key].dtype.str.startswith('|S'):
        meta2[key] = meta2[key].astype('str') 
df2 = meta2.to_pandas()

In [145]:
df = sc2.to_pandas()
df = df[df['Classification'] != 'Duplicate'].copy()
# Remove whitesapce from GRs
df['Classification'] = df['Classification'].str.strip()
df2 = meta2.to_pandas()

In [146]:
rows = len(df)
greenlist = []
for key in df2.columns:
    if key not in df.columns:
        if key not in ('StrayID', 'ObsID', 'MJD', 'Exposure (s)'):
            print(key)
            greenlist = np.append(greenlist, key)
            newcol = [-999.0 for x in range(rows)]
            df[key] = newcol

Area (cm2)
Total Counts (3-8 keV)
Error Total Counts (3-8 keV)
Total Counts (8-13 keV)
Error Total Counts (8-13 keV)
HR
Error HR
3-8 keV count/s/cm2
Error 3-8 keV count/s/cm2
8-13 keV count/s/cm2
Error 8-13 keV count/s/cm2
3-8 keV bkgd count/s/cm2
Error 3-8 keV bkgd count/s/cm2
8-13 keV bkgd count/s/cm2
Error 8-13 keV bkgd count/s/cm2


In [147]:
for row in df2.iterrows():
    for key in greenlist:
        df[key].loc[df['STRAYID'] == row[1]['StrayID']] = row[1][key]
    


/Users/bwgref/science/local/src/miniconda3/envs/nustar/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [148]:
tab = Table.from_pandas(df)
tab.write('fits/straycats_v3.fits', overwrite=True)

In [149]:
tab

STRAYID,Classification,SEQID,Module,Exposure,Multi,Primary,TIME,END_TIME,RA_PRIMARY,DEC_PRIMARY,SL Source,SL Type,SIMBAD_ID,RA_SL,DEC_SL,Area (cm2),Total Counts (3-8 keV),Error Total Counts (3-8 keV),Total Counts (8-13 keV),Error Total Counts (8-13 keV),HR,Error HR,3-8 keV count/s/cm2,Error 3-8 keV count/s/cm2,8-13 keV count/s/cm2,Error 8-13 keV count/s/cm2,3-8 keV bkgd count/s/cm2,Error 3-8 keV bkgd count/s/cm2,8-13 keV bkgd count/s/cm2,Error 8-13 keV bkgd count/s/cm2
str15,str7,int64,str1,float64,str1,str23,float64,float64,float64,float64,str23,str16,str23,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
StrayCatsI_0,Faint,30001014002,B,88472.3252,N,IC10_X1,56967.76813870371,56969.70911092593,5.0742,59.2736,??,??,NA,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
StrayCatsI_1,Unkn,90101010002,A,38102.7867,N,IGR_J00291p5934,57231.53203916667,57232.32370583334,7.2752,59.5982,??,??,NA,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
StrayCatsI_2,SL,90201030002,A,54920.5708,N,SWIFT_J003233d6m7306,57586.71606694445,57587.60842805556,8.1969,-73.0956,SMC X-1,HMXB-NS,SMC X-1,19.271436249999994,-73.4433375,0.6867,996.0,31.56,509.0,22.56,0.511,0.028,0.026409,0.000837,0.013496,0.000598,0.0111,0.00555,0.0053,0.00265
StrayCatsI_3,SL,90201041002,A,38601.5531,N,SMC_X3,57704.740372500004,57705.60842805556,13.0299,-72.4574,SMC X-1,HMXB-NS,SMC X-1,19.271436249999994,-73.4433375,4.3074,4121.0,64.2,2024.0,44.99,0.491,0.013,0.024785,0.000386,0.012173,0.000271,0.0135,0.00675,0.0073,0.00365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
StrayCatsII_101,SL,90701306002,B,44607.4871,N,Swift_182251m135218,59265.96607851852,59267.01121740741,275.7079,-13.883,GX 17+2,LMXB-NS,GX 17+2,274.00578749999994,-14.036283333333333,8.9283,326386.0,571.3,68319.0,261.38,0.209,0.001,0.819511,0.001434,0.17154,0.000656,0.0348,0.0174,0.0099,0.00495
StrayCatsII_102,GR,40701002002,A,97596.2824,N,W50_WESTERN_LOBE,59409.8063562963,59412.2126062963,287.6642,5.025,??,??,NA,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
StrayCatsII_103,GR,40701002002,B,96644.4133,N,W50_WESTERN_LOBE,59409.8063562963,59412.2126062963,287.6642,5.025,??,??,NA,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
StrayCatsII_104,SL,90701336002,B,23583.909,N,EXO_2030p375,59526.14663407407,59526.747328518526,308.0343,37.5922,Cyg X-3,HMXB-BH,Cyg X-3,308.1074166666666,40.957750000000004,0.4433,5948.0,77.12,1367.0,36.97,0.23,0.007,0.568928,0.007377,0.130754,0.003536,0.0859,0.04295,0.0301,0.01505


In [150]:
# Check that it worked
cat = Table.read('fits/straycats_v3.fits', format ='fits')

In [151]:
cat

STRAYID,Classification,SEQID,Module,Exposure,Multi,Primary,TIME,END_TIME,RA_PRIMARY,DEC_PRIMARY,SL Source,SL Type,SIMBAD_ID,RA_SL,DEC_SL,Area (cm2),Total Counts (3-8 keV),Error Total Counts (3-8 keV),Total Counts (8-13 keV),Error Total Counts (8-13 keV),HR,Error HR,3-8 keV count/s/cm2,Error 3-8 keV count/s/cm2,8-13 keV count/s/cm2,Error 8-13 keV count/s/cm2,3-8 keV bkgd count/s/cm2,Error 3-8 keV bkgd count/s/cm2,8-13 keV bkgd count/s/cm2,Error 8-13 keV bkgd count/s/cm2
bytes15,bytes7,int64,bytes1,float64,bytes1,bytes23,float64,float64,float64,float64,bytes23,bytes16,bytes23,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
StrayCatsI_0,Faint,30001014002,B,88472.3252,N,IC10_X1,56967.76813870371,56969.70911092593,5.0742,59.2736,??,??,NA,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
StrayCatsI_1,Unkn,90101010002,A,38102.7867,N,IGR_J00291p5934,57231.53203916667,57232.32370583334,7.2752,59.5982,??,??,NA,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
StrayCatsI_2,SL,90201030002,A,54920.5708,N,SWIFT_J003233d6m7306,57586.71606694445,57587.60842805556,8.1969,-73.0956,SMC X-1,HMXB-NS,SMC X-1,19.271436249999994,-73.4433375,0.6867,996.0,31.56,509.0,22.56,0.511,0.028,0.026409,0.000837,0.013496,0.000598,0.0111,0.00555,0.0053,0.00265
StrayCatsI_3,SL,90201041002,A,38601.5531,N,SMC_X3,57704.740372500004,57705.60842805556,13.0299,-72.4574,SMC X-1,HMXB-NS,SMC X-1,19.271436249999994,-73.4433375,4.3074,4121.0,64.2,2024.0,44.99,0.491,0.013,0.024785,0.000386,0.012173,0.000271,0.0135,0.00675,0.0073,0.00365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
StrayCatsII_101,SL,90701306002,B,44607.4871,N,Swift_182251m135218,59265.96607851852,59267.01121740741,275.7079,-13.883,GX 17+2,LMXB-NS,GX 17+2,274.00578749999994,-14.036283333333333,8.9283,326386.0,571.3,68319.0,261.38,0.209,0.001,0.819511,0.001434,0.17154,0.000656,0.0348,0.0174,0.0099,0.00495
StrayCatsII_102,GR,40701002002,A,97596.2824,N,W50_WESTERN_LOBE,59409.8063562963,59412.2126062963,287.6642,5.025,??,??,NA,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
StrayCatsII_103,GR,40701002002,B,96644.4133,N,W50_WESTERN_LOBE,59409.8063562963,59412.2126062963,287.6642,5.025,??,??,NA,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
StrayCatsII_104,SL,90701336002,B,23583.909,N,EXO_2030p375,59526.14663407407,59526.747328518526,308.0343,37.5922,Cyg X-3,HMXB-BH,Cyg X-3,308.1074166666666,40.957750000000004,0.4433,5948.0,77.12,1367.0,36.97,0.23,0.007,0.568928,0.007377,0.130754,0.003536,0.0859,0.04295,0.0301,0.01505
